<b>LIBRERIAS</b>

In [2]:
import gzip
import pandas as pd
import os
import json
import numpy as np
import ast
from textblob import TextBlob
from datetime import datetime

<b>FUNCIONES</b>

In [2]:
def extraccion(name):
    with gzip.open(name, 'rt', encoding= 'utf-8') as f:
        return[ast.literal_eval(line.strip())
               for line in f]

In [2]:
def extracc_loads(name):
    archivo = []
    with gzip.open(name, 'rt', encoding= 'utf-8') as f:
        for line in f: 
                parsed_line = json.loads(line)
                archivo.append(parsed_line) 
    return archivo

<b>EXTRACCIÓN</b>

In [6]:
data_user = extraccion('user_reviews.json.gz')
data_user

NameError: name 'extraccion' is not defined

In [5]:
data_items = extraccion('users_items.json.gz')
data_items

KeyboardInterrupt: 

In [3]:
data_outputs = extracc_loads('steam_games.json.gz')
data_outputs

[{'publisher': nan,
  'genres': nan,
  'app_name': nan,
  'title': nan,
  'url': nan,
  'release_date': nan,
  'tags': nan,
  'reviews_url': nan,
  'specs': nan,
  'price': nan,
  'early_access': nan,
  'id': nan,
  'developer': nan},
 {'publisher': nan,
  'genres': nan,
  'app_name': nan,
  'title': nan,
  'url': nan,
  'release_date': nan,
  'tags': nan,
  'reviews_url': nan,
  'specs': nan,
  'price': nan,
  'early_access': nan,
  'id': nan,
  'developer': nan},
 {'publisher': nan,
  'genres': nan,
  'app_name': nan,
  'title': nan,
  'url': nan,
  'release_date': nan,
  'tags': nan,
  'reviews_url': nan,
  'specs': nan,
  'price': nan,
  'early_access': nan,
  'id': nan,
  'developer': nan},
 {'publisher': nan,
  'genres': nan,
  'app_name': nan,
  'title': nan,
  'url': nan,
  'release_date': nan,
  'tags': nan,
  'reviews_url': nan,
  'specs': nan,
  'price': nan,
  'early_access': nan,
  'id': nan,
  'developer': nan},
 {'publisher': nan,
  'genres': nan,
  'app_name': nan,
  't

<b>TRANSFORMACIÓN Y CARGA</b>

In [5]:
data_user = pd.DataFrame(data_user)
data_items = pd.DataFrame(data_items)
data_outputs = pd.DataFrame(data_outputs)

NameError: name 'data_user' is not defined

In [ ]:
data_user.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [ ]:
col_desanidada = data_user['reviews'].explode()
col_desanidada = pd.DataFrame(col_desanidada)
col_desanidada = pd.json_normalize(col_desanidada['reviews'])
data_user = pd.concat([data_user.drop('reviews', axis=1), col_desanidada], axis=1)

In [ ]:
data_user.drop_duplicates(inplace=True)
data_user.dropna(inplace=True)

In [ ]:
data_user['posted'] = data_user['posted'].str.replace('Posted ', '') #para poder aplicar formato fecha
data_user = data_user.dropna(subset='posted_year')
data_user['posted_year'] = pd.to_datetime(data_user['posted']).dt.year
data_user['posted_year'] = data_user['posted_year'].astype(dtype = 'int64', errors='raise')

In [ ]:
col_desanidada1 = data_items['items'].explode()
col_desanidada1 = pd.DataFrame(col_desanidada1)
col_desanidada1 = pd.json_normalize(col_desanidada1['items'])
data_items = pd.concat([data_items.drop('items', axis=1), col_desanidada1], axis=1)

In [ ]:
data_items.dropna(inplace=True)

In [ ]:
data_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88176 entries, 0 to 88309
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           88176 non-null  object 
 1   items_count       88176 non-null  float64
 2   steam_id          88176 non-null  object 
 3   user_url          88176 non-null  object 
 4   item_id           88176 non-null  object 
 5   item_name         88176 non-null  object 
 6   playtime_forever  88176 non-null  float64
 7   playtime_2weeks   88176 non-null  float64
dtypes: float64(3), object(5)
memory usage: 6.1+ MB


In [ ]:
data_items.to_csv('datasets/data_items.csv', encoding='utf-8', sep=',', index=False)

In [8]:
data_outputs.dropna(inplace=True)

In [ ]:
data_outputs['release_date'] = pd.to_datetime(data_outputs['release_date'], format= 'mixed', errors='coerce')
data_outputs['release_date'] = pd.to_datetime(data_outputs['release_date']).dt.strftime('%Y-%m-%d')
data_outputs['release_year'] = pd.to_datetime(data_outputs['release_date']).dt.year
data_outputs['release_year'] = data_outputs['release_year'].astype('int64', errors='raise')

In [ ]:
data_outputs = data_outputs.dropna(subset='release_year')

In [ ]:
data_outputs.rename(columns={ 'id':'item_id'}, inplace=True)
data_items.dropna(subset='item_id', inplace=True)

In [9]:
data_outputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     22530 non-null  object
 1   genres        22530 non-null  object
 2   app_name      22530 non-null  object
 3   title         22530 non-null  object
 4   url           22530 non-null  object
 5   release_date  22530 non-null  object
 6   tags          22530 non-null  object
 7   reviews_url   22530 non-null  object
 8   specs         22530 non-null  object
 9   price         22530 non-null  object
 10  early_access  22530 non-null  object
 11  id            22530 non-null  object
 12  developer     22530 non-null  object
dtypes: object(13)
memory usage: 2.4+ MB


In [10]:
data_outputs.to_csv('datasets/data_outputs.csv', sep=',', encoding='utf-8', index=False)

<b>FEATURE ENGINEERING</b>

In [ ]:
def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    if analysis.sentiment.polarity > 0:
        return '2' #positivo
    elif analysis.sentiment.polarity == 0:
        return '1' #neutral
    elif analysis.sentiment.polarity < 0:
        return '0' #negativo
    else: 
        return '1'

In [ ]:
data_user['sentiment_analysis'] = data_user['review'].apply(analizar_sentimiento)

In [ ]:
data_user = data_user.drop(columns= 'review')

In [ ]:
data_user.to_csv('datasets/data_user.csv', encoding='utf-8', sep=',', index=False) #Generé el archivo nuevamente para incluir la columna de analisis de sentimientos.

<b>DATASETS PARA FUNCIONES</b>

In [3]:
data_items=pd.read_csv('data_items.csv')
data_user=pd.read_csv('data_user.csv')
data_steam=pd.read_csv('data_outputs.csv')

PLAYTIMEGENRE

In [7]:
df = data_items.merge(data_steam, on='item_id', how='inner')
df = df.merge(data_user, on='item_id', how='inner')

In [8]:
df.dropna(inplace=True)

In [11]:
columnas=['genres', 'posted_year', 'playtime_forever']
df_playtimegenre = df[columnas]

In [14]:
df_playtimegenre['playtime_forever'] = df_playtimegenre['playtime_forever'].replace(0.0, pd.NA)
df_playtimegenre = df_playtimegenre.dropna()

C:\Users\rebe\AppData\Local\Temp\ipykernel_10788\132596756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playtimegenre['playtime_forever'] = df_playtimegenre['playtime_forever'].replace(0.0, pd.NA)


In [16]:
df_playtimegenre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78655 entries, 2967 to 3353589
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            78655 non-null  object
 1   posted_year       78655 non-null  int64 
 2   playtime_forever  78655 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.4+ MB


In [17]:
df_playtimegenre.to_csv('datasets/df_playtimegenre.csv', sep=',', encoding = 'utf-8', index=False)

USER RECOMMEND

In [43]:
df = data_items.merge(data_user, on='item_id', how='inner')

In [47]:
df.dropna(inplace=True)

In [48]:
columnas = ['item_id','recommend','sentiment_analysis','posted_year','item_name']
df_usersrecommend = df[columnas]

In [49]:
df_usersrecommend.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89982 entries, 2952 to 3852490
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             89982 non-null  int64 
 1   recommend           89982 non-null  bool  
 2   sentiment_analysis  89982 non-null  int64 
 3   posted_year         89982 non-null  int64 
 4   item_name           89982 non-null  object
dtypes: bool(1), int64(3), object(1)
memory usage: 3.5+ MB


In [50]:
df_usersrecommend.to_csv('datasets/df_usersrecommend.csv', sep=',', encoding = 'utf-8', index=False)